# Importing Libraries & Reading JSON

In [1]:
import sys
import os
import json
import itertools

json_list = json.load(open('parameters_json_file.json', 'r'))
print("parameters_json_file.json just opened")

data = json_list["data"]
parameters = json_list["parameters"]
fixed_parameters = parameters["fixed"]
iterable_parameters = parameters["iterable"]
print(iterable_parameters)

parameters_json_file.json just opened
{'word_min_freq': [0, 5], 'char_rnn': [25, 50]}


## Parameters Combinations

In [11]:
listofvalues=iterable_parameters.values()
args_list = []
for it_key in iterable_parameters:
    print(it_key, "\t", iterable_parameters[it_key])
    args_list.append(it_key)
parameter_combinations = list(itertools.product(*listofvalues))
print("\nNumber of combinations to try:", len(parameter_combinations))


word_min_freq 	 [0, 5]
char_rnn 	 [25, 50]

Number of combinations to try: 4


## Class: Command - add_arg()

In [12]:
class Command_string:
    def __init__(self, initial_string = "python train.py"):
        self.string = initial_string
    
    def change(self, new_string):
        self.string = new_string
        
    def add_arg_dic(self, key, dic):
        self.string += " --" + key + " " + str(dic[key])
    
    def add_arg_val(self, key, val):
        self.string += " --" + key + " " + str(val)

## Commands String

### Fixed string part

In [13]:
fixed_command = Command_string()

for path_key in data:
    fixed_command.add_arg_dic(path_key, data)
for fixed_par_key in fixed_parameters:
    fixed_command.add_arg_dic(fixed_par_key, fixed_parameters)
print(fixed_command.string)

python train.py --train_path train.bilou_CRF --val_path validation.bilou_CRF --test_path test.bilou_CRF --data_path data/path --emb_path data/pretrained_embedding.txt --max_epochs 100 --char_emb_dim 25 --word_emb_dim 100


### Changing part (due to arguments combinations) --> list of strings

In [16]:
commands_list = []
tmp_command = Command_string()
#print(tmp_command.string)

for comb in parameter_combinations:
    tmp_command.change(fixed_command.string)
    comb_num = parameter_combinations.index(comb)
    name = "training_" + str(comb_num) + "_"
    for it_par_key in iterable_parameters:
        key_index = list(iterable_parameters.keys()).index(it_par_key)
        tmp_command.add_arg_val(it_par_key, comb[key_index])
        #print(key_index, it_par_key, comb[key_index])
        name += "__" + it_par_key + "_" + str(comb[key_index])
        checkpoint_dir = json_list["checkpoints"] + "/" + name
        tmp_command.add_arg_val("checkpoint", checkpoint_dir)
    #print("--> \n", tmp_command.string, "\n")
    print(name)
    post_command = " | tee -a " + checkpoint_dir + "/" + name + ".log"
    commands_list.append(tmp_command.string + post_command)
print("List length:", len(commands_list))
index_to_show = 1
print("List command %d (%d/%d):\n%s" % (index_to_show, index_to_show+1, len(commands_list), commands_list[index_to_show]))

training_0___word_min_freq_0__char_rnn_25
training_1___word_min_freq_0__char_rnn_50
training_2___word_min_freq_5__char_rnn_25
training_3___word_min_freq_5__char_rnn_50
List length: 4
List command 1 (2/4):
python train.py --train_path train.bilou_CRF --val_path validation.bilou_CRF --test_path test.bilou_CRF --data_path data/path --emb_path data/pretrained_embedding.txt --max_epochs 100 --char_emb_dim 25 --word_emb_dim 100 --word_min_freq 0 --checkpoint /disks/md0-4T/users/b.mgm/checkpoints/training_1___word_min_freq_0 --char_rnn 50 --checkpoint /disks/md0-4T/users/b.mgm/checkpoints/training_1___word_min_freq_0__char_rnn_50 | tee -a /disks/md0-4T/users/b.mgm/checkpoints/training_1___word_min_freq_0__char_rnn_50/training_1___word_min_freq_0__char_rnn_50.log


# Output File

## Opening File

In [17]:
print("Opening writing file")
out_file_name = json_list["output_name"]
print("Name of file:", out_file_name)
out_file = open(out_file_name, 'w') # Passar a 'x' per a que creï un nou fitxer i sinó ERROR

Opening writing file
Name of file: training_commands.lst


## Printing String into File

In [18]:
for line in commands_list:
    out_file.write(line + "\n")
out_file.close()

In [19]:
out_file = open(out_file_name, 'r')
print(out_file.read())
out_file.close()

python train.py --train_path train.bilou_CRF --val_path validation.bilou_CRF --test_path test.bilou_CRF --data_path data/path --emb_path data/pretrained_embedding.txt --max_epochs 100 --char_emb_dim 25 --word_emb_dim 100 --word_min_freq 0 --checkpoint /disks/md0-4T/users/b.mgm/checkpoints/training_0___word_min_freq_0 --char_rnn 25 --checkpoint /disks/md0-4T/users/b.mgm/checkpoints/training_0___word_min_freq_0__char_rnn_25 | tee -a /disks/md0-4T/users/b.mgm/checkpoints/training_0___word_min_freq_0__char_rnn_25/training_0___word_min_freq_0__char_rnn_25.log
python train.py --train_path train.bilou_CRF --val_path validation.bilou_CRF --test_path test.bilou_CRF --data_path data/path --emb_path data/pretrained_embedding.txt --max_epochs 100 --char_emb_dim 25 --word_emb_dim 100 --word_min_freq 0 --checkpoint /disks/md0-4T/users/b.mgm/checkpoints/training_1___word_min_freq_0 --char_rnn 50 --checkpoint /disks/md0-4T/users/b.mgm/checkpoints/training_1___word_min_freq_0__char_rnn_50 | tee -a /dis

In [123]:
def add_parameter(string, key, dic):
    return command + " --" + key + " " + str(dic[key])

In [124]:
command_string = "python train.py"
#for comb in parameter_combinations:
for path_key in data:
    command_string = add_parameter(string=command_string, key=path_key, dic=path)
for fixed_par_key in fixed_parameters:
    command_string = add_parameter(string=command_string, key=fixed_par_key, dic=fixed_parameters)
for comb in parameter_combinations:
    for it_par_key in iterable_parameters:
        command_string = add_parameter(string=command_string)
print(command_string)

python train.py --data_path data/path --train_path train.bilou_CRF --val_path validation.bilou_CRF --test_path test.bilou_CRF --emb_path data/pretrained_embedding.txt --max_epochs 100 --char_emb_dim 25 --word_emb_dim 100
